In [21]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import ast
import datetime as dt
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import NASNetMobile, MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.models import load_model
from joblib import Parallel, delayed
from functools import partial
import time
start = dt.datetime.now()

In [2]:
DP_DIR = ''
INPUT_DIR = ''

BASE_SIZE = 256
NCSVS = 100
NCATS = 340
np.random.seed(seed=1987)
tf.set_random_seed(seed=1987)

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, '../../data/train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)

In [3]:
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [ ]:
#The number of images originially used by the kernel
#I keep it the same for fair comparison 
MAGIC_CONSTANT = 800 * 680  
batchsize = 2048
STEPS = MAGIC_CONSTANT//batchsize 
EPOCHS = 16
size = 64



In [5]:
# model = NASNetMobile(input_shape=(size, size, 1), weights=None, classes=NCATS)
model = MobileNet(input_shape=(size, size, 1), alpha=1., weights=None, classes=NCATS)
model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 65, 65, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 32, 32, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 32, 32, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 32, 32, 32)        128       
__________

In [42]:
def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                         (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

def get_input(df, size, lw, time_color):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw,
                                 time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    y = keras.utils.to_categorical(df.y, num_classes=NCATS)
    return x, y

    
def image_generator_xd(size, batchsize, ks, lw=6, time_color=True):
    partial_get_input = partial(get_input, size = size, lw=lw, time_color=time_color)
    while True:
        for k in np.random.permutation(ks):
            filename = os.path.join(DP_DIR, 'full_train/train_k{}.csv.gz'.format(k))
            chunks = list(pd.read_csv(filename, chunksize=batchsize))
            batches = Parallel(n_jobs=-1)(delayed(partial_get_input)(chunk) for chunk in chunks)
            for batch in batches:
                yield batch 
                
def df_to_image_array_xd(df, size, lw=6, time_color=True):
    df['drawing'] = df['drawing'].apply(ast.literal_eval)
    x = np.zeros((len(df), size, size, 1))
    for i, raw_strokes in enumerate(df.drawing.values):
        x[i, :, :, 0] = draw_cv2(raw_strokes, size=size, lw=lw, time_color=time_color)
    x = preprocess_input(x).astype(np.float32)
    return x



In [7]:
valid_df = pd.read_csv(os.path.join(DP_DIR, 'train_k{}.csv.gz'.format(NCSVS - 1)), nrows=34000)
x_valid = df_to_image_array_xd(valid_df, size)
y_valid = keras.utils.to_categorical(valid_df.y, num_classes=NCATS)
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 64, 64, 1) (34000, 340)
Validation array memory 0.52 GB


In [8]:
#model.load_weights("model")

In [43]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, ks=range(NCSVS - 1))

In [38]:
start = time.time()
for i in range(100):
    a = next(train_datagen)
print(time.time()-start)

22.50601053237915


22.1

51.83713412284851


In [39]:
callbacks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    ModelCheckpoint(filepath='model', verbose=1, save_best_only=True)
]
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/16
264/265 [============================>.] - ETA: 1s - loss: 1.0317 - categorical_crossentropy: 1.0317 - categorical_accuracy: 0.7351 - top_3_accuracy: 0.8877
Epoch 00001: val_loss improved from inf to 1.12168, saving model to model
265/265 [==============================] - 324s 1s/step - loss: 1.0316 - categorical_crossentropy: 1.0316 - categorical_accuracy: 0.7351 - top_3_accuracy: 0.8877 - val_loss: 1.1217 - val_categorical_crossentropy: 1.1217 - val_categorical_accuracy: 0.7151 - val_top_3_accuracy: 0.8754
Epoch 2/16
264/265 [============================>.] - ETA: 1s - loss: 1.0231 - categorical_crossentropy: 1.0231 - categorical_accuracy: 0.7372 - top_3_accuracy: 0.8894
Epoch 00002: val_loss did not improve from 1.12168
265/265 [==============================] - 324s 1s/step - loss: 1.0231 - categorical_crossentropy: 1.0231 - categorical_accuracy: 0.7371 - top_3_accuracy: 0.8894 - val_loss: 3.1170 - val_categorical_crossentropy: 3.1170 - val_categorical_accuracy: 0.3736 

In [40]:
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/16
264/265 [============================>.] - ETA: 1s - loss: 0.8649 - categorical_crossentropy: 0.8649 - categorical_accuracy: 0.7755 - top_3_accuracy: 0.9113
Epoch 00001: val_loss did not improve from 0.89437
265/265 [==============================] - 361s 1s/step - loss: 0.8647 - categorical_crossentropy: 0.8647 - categorical_accuracy: 0.7755 - top_3_accuracy: 0.9113 - val_loss: 0.8982 - val_categorical_crossentropy: 0.8982 - val_categorical_accuracy: 0.7656 - val_top_3_accuracy: 0.9060
Epoch 2/16
264/265 [============================>.] - ETA: 1s - loss: 0.8593 - categorical_crossentropy: 0.8593 - categorical_accuracy: 0.7768 - top_3_accuracy: 0.9116
Epoch 00002: val_loss improved from 0.89437 to 0.89432, saving model to model
265/265 [==============================] - 323s 1s/step - loss: 0.8593 - categorical_crossentropy: 0.8593 - categorical_accuracy: 0.7768 - top_3_accuracy: 0.9115 - val_loss: 0.8943 - val_categorical_crossentropy: 0.8943 - val_categorical_accuracy: 0.7

In [ ]:
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbacks
)
hists.append(hist)

Epoch 1/16
264/265 [============================>.] - ETA: 2s - loss: 0.8155 - categorical_crossentropy: 0.8155 - categorical_accuracy: 0.7888 - top_3_accuracy: 0.9173
Epoch 00001: val_loss did not improve from 0.81882
265/265 [==============================] - 556s 2s/step - loss: 0.8155 - categorical_crossentropy: 0.8155 - categorical_accuracy: 0.7887 - top_3_accuracy: 0.9173 - val_loss: 0.8262 - val_categorical_crossentropy: 0.8262 - val_categorical_accuracy: 0.7862 - val_top_3_accuracy: 0.9158
Epoch 2/16
264/265 [============================>.] - ETA: 1s - loss: 0.8229 - categorical_crossentropy: 0.8229 - categorical_accuracy: 0.7875 - top_3_accuracy: 0.9162
Epoch 00002: val_loss did not improve from 0.81882
265/265 [==============================] - 338s 1s/step - loss: 0.8227 - categorical_crossentropy: 0.8227 - categorical_accuracy: 0.7876 - top_3_accuracy: 0.9162 - val_loss: 0.8210 - val_categorical_crossentropy: 0.8210 - val_categorical_accuracy: 0.7869 - val_top_3_accuracy: 0

In [ ]:
hist_df = pd.concat([pd.DataFrame(hist.history) for hist in hists], sort=True)
hist_df.index = np.arange(1, len(hist_df)+1)
fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
axs[0].plot(hist_df.val_categorical_accuracy, lw=5, label='Validation Accuracy')
axs[0].plot(hist_df.categorical_accuracy, lw=5, label='Training Accuracy')
axs[0].set_ylabel('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].grid()
axs[0].legend(loc=0)
axs[1].plot(hist_df.val_categorical_crossentropy, lw=5, label='Validation MLogLoss')
axs[1].plot(hist_df.categorical_crossentropy, lw=5, label='Training MLogLoss')
axs[1].set_ylabel('MLogLoss')
axs[1].set_xlabel('Epoch')
axs[1].grid()
axs[1].legend(loc=0)
fig.savefig('hist.png', dpi=300)
plt.show();

In [ ]:
valid_predictions = model.predict(x_valid, batch_size=128, verbose=1)
map3 = mapk(valid_df[['y']].values, preds2catids(valid_predictions).values)
print('Map3: {:.3f}'.format(map3))

In [ ]:
test = pd.read_csv(os.path.join(INPUT_DIR, 'test_simplified.csv'))
test.head()
x_test = df_to_image_array_xd(test, size)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

In [ ]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv('gs_mn_submission_{}.csv'.format(int(map3 * 10**4)), index=False)
submission.head()
submission.shape

In [ ]:
end = dt.datetime.now()
print('Latest run {}.\nTotal time {}s'.format(end, (end - start).seconds))